In [1]:
%matplotlib inline
import math
import pandas as pd

In [38]:
df = pd.read_csv('./dataset/transformed_train.csv')
df.describe()

,Yearly_ExpensesK,Avg_Residancy_Altitude,Overall_happiness_score,Will_vote_only_large_party,Financial_agenda_matters,Yearly_IncomeK,Most_Important_Issue,Vote
count,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000
mean,0.002270,-0.006917,0.008513,0.958000,0.186167,0.011338,3.097833,5.108833
std,1.000050,0.988772,0.999945,0.200606,0.389274,0.990657,2.408782,3.360735
min,-2.945120,-2.432355,-3.726302,0.000000,0.000000,-3.918231,0.000000,0.000000
25%,-0.601923,-0.805058,-0.625250,1.000000,0.000000,-0.487121,1.000000,2.000000
50%,-0.349992,-0.234427,-0.058003,1.000000,0.000000,-0.218022,3.000000,5.000000
75%,0.517605,0.606814,0.619365,1.000000,0.000000,0.316993,5.000000,9.000000
max,2.868502,3.344068,2.473634,1.000000,1.000000,3.091690,7.000000,9.000000


In [ ]:
def get_votes_count_by_party(df):
    parties = {}
    for party in df.Vote.unique():
        parties[party] = len(df[df.Vote == party])
    return parties;

In [ ]:
from sklearn.cluster import KMeans
n_clusters = 100;
cls = KMeans(n_clusters=n_clusters)
res = cls.fit_predict(df.drop('Vote', axis=1))

for v in xrange(n_clusters):
    sum_ = sum(get_votes_count_by_party(df.iloc[[x for x,y in enumerate(res) if y==v]]).itervalues())
    print max({k: (v + .0) / sum_ for k, v in get_votes_count_by_party(df.iloc[[x for x,y in enumerate(res) if y==v]]).iteritems()}.itervalues())

In [ ]:
ones = [x for x,y in enumerate(res) if y==1]
zeros = [x for x,y in enumerate(res) if y==0]
f = [x for x,y in enumerate(res) if y==3]

In [ ]:
zeros_parties = get_votes_count_by_party(df.iloc[zeros])
f_parties = get_votes_count_by_party(df.iloc[f])
ones_parties = get_votes_count_by_party(df.iloc[ones])
all_parties = get_votes_count_by_party(df)

In [ ]:
{k: (v + .0) / sum_ for k, v in f_parties.iteritems()}

In [13]:
minimal_coalitions = []
def is_majority(coalition):    
    return len(df[df.Vote.isin(coalition)]) > len(df)/2

def is_minimal_coalition(coalition):
    if not is_majority(coalition):
        return False
    for party in coalition:
        if (is_majority(coalition.difference([party]))):
            return False
    return True

def get_coalitions(coalition, i):
    if is_minimal_coalition(coalition):
        minimal_coalitions.append(coalition)
    else:
        if i==10:
            return
        get_coalitions(coalition.union([i]), i+1)
        get_coalitions(coalition, i+1)

coalition = set()
get_coalitions(coalition, 0)

In [14]:
minimal_coalitions

[{0, 1, 2, 3, 4, 5},
 {0, 1, 2, 3, 4, 8},
 {0, 1, 2, 4, 7},
 {0, 1, 2, 5, 7, 8},
 {0, 1, 3, 4, 5, 8},
 {0, 1, 3, 6, 7, 8},
 {0, 1, 4, 5, 7},
 {0, 1, 4, 7, 8},
 {0, 1, 5, 9},
 {0, 2, 3, 4, 5, 8},
 {0, 2, 3, 4, 6, 8},
 {0, 2, 3, 5, 7},
 {0, 2, 3, 7, 8},
 {0, 2, 4, 5, 7},
 {0, 2, 4, 7, 8},
 {0, 2, 9},
 {0, 3, 4, 7},
 {0, 3, 5, 7, 8},
 {0, 3, 9},
 {0, 4, 5, 7, 8},
 {0, 4, 9},
 {0, 5, 6, 9},
 {0, 7, 9},
 {0, 8, 9},
 {1, 2, 3, 5, 9},
 {1, 2, 3, 8, 9},
 {1, 2, 4, 5, 9},
 {1, 2, 7, 9},
 {1, 3, 4, 9},
 {1, 3, 5, 8, 9},
 {1, 5, 6, 7, 9},
 {1, 7, 8, 9},
 {2, 3, 4, 5, 7, 8},
 {2, 3, 4, 9},
 {2, 3, 5, 8, 9},
 {2, 4, 5, 6, 9},
 {2, 4, 8, 9},
 {2, 5, 7, 9},
 {2, 7, 8, 9},
 {3, 4, 5, 9},
 {3, 4, 8, 9},
 {3, 7, 9},
 {4, 5, 8, 9},
 {4, 7, 9},
 {5, 7, 8, 9}]

In [15]:
len(minimal_coalitions)

45

In [55]:
import numpy
def evaluate_coalition(df, coalition):
    coalition_df = df[df.Vote.isin(coalition)].drop('Vote',axis=1)
    coalition_mean = coalition_df.mean()
    oposition_mean = df[-df.Vote.isin(coalition)].drop('Vote',axis=1).mean()
    coalition_df['distance_from_mean'] = numpy.linalg.norm(coalition_df-coalition_mean, axis=1)    
    distance_within_coalition = coalition_df['distance_from_mean'].mean()
    distance_between = numpy.linalg.norm(coalition_mean-oposition_mean)
    score = distance_between / distance_within_coalition
    print 'Coalition: ' + str(coalition) + ", size: " + str(len(coalition_df)) + ", score: " + str(score)
    
for coalition in minimal_coalitions:
    evaluate_coalition(df.drop('Most_Important_Issue', axis=1), coalition)

Coalition: set([0, 1, 2, 3, 4, 5]), size: 3116, score: 0.336893585984
Coalition: set([0, 1, 2, 3, 4, 8]), size: 3163, score: 0.233995084843
Coalition: set([0, 1, 2, 4, 7]), size: 3044, score: 0.213518587919
Coalition: set([0, 1, 2, 5, 7, 8]), size: 3096, score: 0.351120155127
Coalition: set([0, 1, 3, 4, 5, 8]), size: 3141, score: 0.143579220627
Coalition: set([0, 1, 3, 6, 7, 8]), size: 3010, score: 0.186150585191
Coalition: set([0, 1, 4, 5, 7]), size: 3022, score: 0.181416151593
Coalition: set([0, 1, 4, 8, 7]), size: 3069, score: 0.240800531484
Coalition: set([0, 1, 5, 9]), size: 3200, score: 0.56172749155
Coalition: set([0, 2, 3, 4, 5, 8]), size: 3295, score: 0.368156300995
Coalition: set([0, 2, 3, 4, 6, 8]), size: 3015, score: 0.246346411923
Coalition: set([0, 2, 3, 5, 7]), size: 3065, score: 0.355164551627
Coalition: set([0, 8, 2, 3, 7]), size: 3112, score: 0.32112260258
Coalition: set([0, 2, 4, 5, 7]), size: 3176, score: 0.355349019112
Coalition: set([0, 8, 2, 4, 7]), size: 3223, s

In [42]:
coalition_df

NameError: name 'coalition_df' is not defined

In [33]:
df = pd.DataFrame([[1,2],[3,4]],columns=["a","b"])

In [35]:
df.mean()

a    2
b    3
dtype: float64

In [36]:
df-df.mean()

,a,b
0,-1,-1
1,1,1
